This notebook demonstrates a simple workflow to tackle the MM&M Supermarket case.

In [1]:
# load python packages for data processing and analysis
import pandas as pd

# 1 A quick look at the data

In [2]:
# load the data
mma_mart = pd.read_csv('mma_mart.csv')

In [3]:
mma_mart.head()

,order_id,product_id,product_name,aisle_id,aisle,department_id,department
0,1,49302,Bulgarian Yogurt,120,yogurt,16,dairy eggs
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs
2,1,10246,Organic Celery Hearts,83,fresh vegetables,4,produce
3,1,49683,Cucumber Kirby,83,fresh vegetables,4,produce
4,1,43633,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods


In [4]:
mma_mart.shape

(987259, 7)

In [5]:
mma_mart.nunique()

order_id         97833
product_id       35070
product_name     35070
aisle_id           134
aisle              134
department_id       21
department          21
dtype: int64

The dataset has about 987K rows. Since we have about 98K unique orders, on average each order contains about 10 products. We have about 35K unique products, and they are from 134 aisles, and 21 departments.

You perhaps want to do more exploratory data analysis (EDA), but I will move on to pick 1000 products for the special Instabasket aisle.

# 2 Pick 1000 products

I will use a subset of the data to build a model (i.e., to run some analysis) to pick 1000 products. The remaining data will be used to evaluate the model performance (i.e., to calculate the performance metrics).

In [6]:
train = mma_mart[mma_mart['order_id'] <= 80000]
test = mma_mart[mma_mart['order_id'] > 80000]

Note the training data doesn't contain all the possible products. That's OK. In the real world it's likely that new products are constantly introduced in the MM&A Supermarket, so our choice of the 1000 products doesn't have to be based on data that contain all products.

In [7]:
train.nunique()

order_id         78249
product_id       33171
product_name     33171
aisle_id           134
aisle              134
department_id       21
department          21
dtype: int64

Next, I build a simple model to pick 1000 products. Indeed, the model is so simple that it just randomly pick 1000 products.

In [8]:
# keep the 'aisle_id', 'department_id' columns for later use
my_pick = train[['product_id', 'product_name', 'aisle_id', 'department_id']].drop_duplicates().sample(n=1000, random_state=2)

In [9]:
my_pick.head()

,product_id,product_name,aisle_id,department_id
218611,16967,Organic 50/50 Spring Blend,123,4
121171,47771,100% Organic Coconut Water,98,7
314635,10947,No Sugar Added Tapioca Pudding,71,16
541290,6189,Meat Snacks Sriracha Beef Jerky,23,19
320169,15188,Grands Homestyle Honey Butter Buscuits,105,13


Let's check whether my pick satisifies the contraint that no more than 100 frozen products and 100 refrigerated products can be in the special aisle.

In [10]:
# list all department to identify frozen and refrigerated products
mma_mart[['department_id', 'department']].drop_duplicates().sort_values(by='department_id')

,department_id,department
68,1,frozen
691,2,other
24,3,bakery
2,4,produce
120,5,alcohol
45,6,international
35,7,beverages
119,8,pets
48,9,dry goods pasta
154,10,bulk


After manually checking all the departments, I assume items from the "frozen" department are frozen products, and items from the "meat seafood" and "dairy eggs" departments are refrigerated products.

**Note: I only use department names to identify frozen and refrigerated products. You should probably do a better job by using the aisle information (or even product names) too.**

In [11]:
# count frozen products
my_pick[my_pick['department_id'] == 1]['department_id'].count()

90

In [12]:
# refrigerated products
my_pick[(my_pick['department_id'] == 12) | (my_pick['department_id'] == 16)]['department_id'].count()

99

Our pick satisfies the constraint.

# 3 Calculate the metrics

Let's calculate the two metrics to check our model performance.

(1) the number of orders that utilize the in-aisle items (and compared it with the total number of orders)

(2) the average number of items in each order that utilize in-aisle items (and compare it with the average number of items in each order)

We will use the **test data**, and calcuate the metrics with and without accounting for the substitutes.

## 3.1 Calculate the metrics without accounting for the substitutes

In [13]:
# calculate total unique orders in the test data
tot_order = test['order_id'].nunique()
print(f"Total number of orders: {tot_order}")

Total number of orders: 19584


In [14]:
# calculate the average number of items in each order
avg_item = len(test) / tot_order
print(f"Average number of items in each order: {avg_item}")

Average number of items in each order: 10.136540032679738


In [15]:
# join the test data with my pick on 'product_id'
test_my_pick = test.merge(my_pick, on='product_id', how='left')

# group by 'order_id' and count the non-null values of 'product_name_y' column
metrics_tmp = test_my_pick[['order_id', 'product_name_y']].groupby('order_id').count().reset_index().rename(columns={'product_name_y': "num_prod_matching"})
metrics_tmp.head()

,order_id,num_prod_matching
0,80001,0
1,80002,0
2,80003,0
3,80004,1
4,80005,0


In [16]:
# calculate metric 1
# count orders with at least a match
metric_1 = len(metrics_tmp[metrics_tmp['num_prod_matching']>0])
print(f"Out of {tot_order} orders, {metric_1} of them utilize the in-aisle items. That's about {metric_1/tot_order*100:.2f}%.")

Out of 19584 orders, 7510 of them utilize the in-aisle items. That's about 38.35%.


In [17]:
# calculate metric 2
metric_2 = metrics_tmp['num_prod_matching'].sum() / len(metrics_tmp)
print(f"On average there are {avg_item:.2f} items in each order, and {metric_2:.2f} of them utilize in-aisle items. That's about {metric_2/avg_item*100:.2f}%")


On average there are 10.14 items in each order, and 0.53 of them utilize in-aisle items. That's about 5.21%


The resulting metrics don't seem to be too good.

## 3.2 Calculate the metrics accounting for the substitutes

I assume products/items from the same department and same aisle are substitutes.

**Note: This is an oversimplified assumption. You should do better here.**

In [18]:
# find unique 'aisle_id' and 'department_id' in my_pick
my_aisle_dept = my_pick[['aisle_id', 'department_id']].drop_duplicates()

# add a new column for identifying non-matching rows later
my_aisle_dept["special_aisle"] = True
my_aisle_dept.head()

,aisle_id,department_id,special_aisle
218611,123,4,True
121171,98,7,True
314635,71,16,True
541290,23,19,True
320169,105,13,True


In [19]:
# join the test data with my_aisle_dept on 'aisle_id' and 'department_id'
test_my_aisle_dept = test.merge(my_aisle_dept, on=['aisle_id', 'department_id'], how='left')

In [20]:
# group by 'order_id' and count the non-null values of 'product_name_y' column
metrics_sub_tmp = test_my_aisle_dept[['order_id', 'special_aisle']].groupby('order_id').count().reset_index().rename(columns={'special_aisle': "num_prod_matching"})
metrics_sub_tmp.head()

,order_id,num_prod_matching
0,80001,7
1,80002,8
2,80003,14
3,80004,5
4,80005,4


In [21]:
# calculate metric 1
# count orders with at least a match
metric_sub_1 = len(metrics_sub_tmp[metrics_sub_tmp['num_prod_matching']>0])
print(f"Out of {tot_order} orders, {metric_sub_1} of them utilize the in-aisle items. That's about {metric_sub_1/tot_order*100:.2f}%.")

Out of 19584 orders, 19572 of them utilize the in-aisle items. That's about 99.94%.


In [22]:
# calcualte metric 2
metric_sub_2 = metrics_sub_tmp['num_prod_matching'].sum() / len(metrics_sub_tmp)
print(f"On average there are {avg_item:.2f} items in each order, and {metric_sub_2:.2f} of them utilize in-aisle items. That's about {metric_sub_2/avg_item*100:.2f}%")

On average there are 10.14 items in each order, and 9.95 of them utilize in-aisle items. That's about 98.18%


The resulting metrics look too good. Our assumption on substitutes is oversimplied and likely wrong.